In [1]:
# !pip install open3d
import open3d as o3d
import numpy as np
import sys

sys.path.append('./data/')
from camera import *
from dataloader import *

sys.path.append('./algorithm/')
from measurement import *
from pose import *
from update import *
from prediction import *

from pipeline import *

import matplotlib.pyplot as plt
import time

In [ ]:
vbg = o3d.t.geometry.VoxelBlockGrid.load('./results/TSDF_1050.npz')
visualize_vbg_o3d(vbg, 180, 0, 0)

PointCloud on CPU:0 [3607538 points (Float32)].
Attributes: normals (dtype = Float32, shape = {3607538, 3}).


[error] GLFW error: Cocoa: Failed to find service port for display
[error] GLFW error: Cocoa: Failed to find service port for display
[error] GLFW error: Cocoa: Failed to find service port for display
[error] GLFW error: Cocoa: Failed to find service port for display
[error] GLFW error: Cocoa: Failed to find service port for display
[error] GLFW error: Cocoa: Failed to find service port for display
[error] GLFW error: Cocoa: Failed to find service port for display
